In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ply
import numpy as np

In [ ]:
datos = pd.read_csv("D:/DatasetsVault/HouseSalesKingCounty/kc_house_data.csv")